# Pypsark Dataframe'ini Panda Dataframe'ine çevirirken yaşanan performans sıkıntısına bir çözüm: Pyarrow Modulü 
 <br>
<font size=4,font style=arial>
 Bu yazımızda oluşturduğumuz pyspark dataframe'ini panda daraframe'ine çevirirken yaşanan bekleme süresini %80 <br> 
 iyileştiren bir modül olan Pyarrow modülünü test edeceğiz. Öncelikle Windows Command penceresinde "pip install <br> 
 pyarrow" yazıp bu modülü kurmanız gerekmektedir. <br> 
Not:Kurmadan önce bilgisayarınız 64 bit ve sizde python 32 bit kurulmuşsa bu problem. Python 64 bit için olanı <br>
  kurmanız gerekiyor. Bu düzeltme yapılmazsa Pyarrow paketi kurulmuyor.<br>   
</font>   

<font size=4,font style=arial>
Sparksession ve sparkcontext'i oluşturalım. Round,pandas,randn fonskiyonlarını yükleyelim
</font> 

In [2]:
 #for the round function
import pyspark.sql.functions as func
#for pandas
import pandas as pd
#for randn 
from pyspark.sql.functions import randn
##create spark session
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

<font size=4,font style=arial>
5 tane normal dağılımlı(mean=0,std=1) kolona sahip olan veri setimizi oluşturalım.
</font>  

In [3]:
df1 = sqlContext.range(0, 4000000)\
     .withColumn('normal1',func.abs(func.round(100*randn(seed=1),2)))\
     .withColumn('normal2',func.abs(func.round(100*randn(seed=2),2)))\
     .withColumn('normal3',func.abs(func.round(100*randn(seed=3),2)))\
     .withColumn('normal4',func.abs(func.round(100*randn(seed=4),2)))\
     .withColumn('normal5',func.abs(func.round(100*randn(seed=5),2)))

<font size=4,font style=arial>
Sparkcontext'in içeriği aşağıda ki şekilde görülebilir. Spark UI tıklanırsa çalışılan jop'lar görüleiblir. Master makinem kendi lokalim.
</font>  

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

<font size=4,font style=arial>
df1 dataframe'ini panda dataframe'i yapalım ve süreyi görelim.
</font> 

In [5]:
%time pdf = df1.toPandas()

Wall time: 1min 7s


<font size=4,font style=arial>
Şimdi pyarrow paketini enable yapıp tekrar panda dataframe'ine çevirip süreyi görelim
</font> 

In [6]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [7]:
%time pdf = df1.toPandas()

Wall time: 14.1 s


<font size=4,font style=arial>
Sürede %80'lik bir iyileştirme var ki bu süper bir rakam.
</font> 

<font size=4,font style=arial>
pdf panda dataframe'i df1 ise spark dataframe'idir. Panda'da head, sparkda show ile dataframe görülebilir.
</font> 

In [8]:
pdf.head(5)

,id,normal1,normal2,normal3,normal4,normal5
0,0,103.23,63.46,92.18,75.12,81.92
1,1,25.31,80.91,25.38,120.15,15.78
2,2,69.78,126.63,70.60,134.24,179.92
3,3,20.28,77.50,80.25,57.03,56.39
4,4,70.96,204.73,31.35,108.08,15.75


In [9]:
df1.show(5)

+---+-------+-------+-------+-------+-------+
| id|normal1|normal2|normal3|normal4|normal5|
+---+-------+-------+-------+-------+-------+
|  0| 103.23|  63.46|  92.18|  75.12|  81.92|
|  1|  25.31|  80.91|  25.38| 120.15|  15.78|
|  2|  69.78| 126.63|   70.6| 134.24| 179.92|
|  3|  20.28|   77.5|  80.25|  57.03|  56.39|
|  4|  70.96| 204.73|  31.35| 108.08|  15.75|
+---+-------+-------+-------+-------+-------+
only showing top 5 rows



<font size=4,font style=arial>
pdf panda dataframe'i olduktan sonra panda paketinin esnekliğinden faydalanarak her türlü veri manipulasyonu yapılabilir.
</font> 

In [10]:
pd.set_option('float_format', '{:f}'.format)

<font size=4,font style=arial>
Mesela describe
</font> 

In [11]:
pdf.describe()

,id,normal1,normal2,normal3,normal4,normal5
count,4000000.000000,4000000.000000,4000000.000000,4000000.000000,4000000.000000,4000000.000000
mean,1999999.500000,79.794201,79.789656,79.746779,79.722241,79.714956
std,1154700.682717,60.244889,60.240130,60.216199,60.213637,60.212164
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,999999.750000,31.880000,31.900000,31.890000,31.850000,31.840000
50%,1999999.500000,67.500000,67.500000,67.445000,67.400000,67.390000
75%,2999999.250000,115.090000,115.050000,114.960000,114.930000,114.910000
max,3999999.000000,490.000000,490.000000,500.770000,500.770000,510.480000
